# DNN Canned Estimator

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
MODEL_DIR = './model/iris_DNN'

In [3]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def maybe_download():
  train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
  test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

  return train_path, test_path

def load_data(y_name='Species'):
  """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
  import pandas as pd
  train_path, test_path = maybe_download()

  train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
  train_x, train_y = train, train.pop(y_name)

  test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
  test_x, test_y = test, test.pop(y_name)

  return (train_x, train_y), (test_x, test_y)

In [4]:
def make_train_input_fn(features, labels, batch_size):
  def train_input_fn():
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    
    # Return the dataset.
    return dataset
  return train_input_fn

def make_eval_input_fn(features, labels=None, batch_size=100):
  def eval_input_fn():
    """An input function for evaluation or prediction"""
    if labels is None:
        # No labels, use only features.
        inputs = dict(features)
    else:
        inputs = (dict(features), labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset
  
  return eval_input_fn

## Training

In [5]:
! rm -rf $MODEL_DIR

In [6]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = load_data()

# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [7]:
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_steps = 2000,
    #keep_checkpoint_max = 10,
    log_step_count_steps = 500,
    tf_random_seed=2020         # for reproducibility
)

# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
  feature_columns=my_feature_columns,
  # Two hidden layers of 10 nodes each.
  hidden_units=[10, 10],
  # The model must choose between 3 classes.
  n_classes=3,
  model_dir=MODEL_DIR,
  config=my_checkpointing_config
)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9e72d9df50>, '_evaluation_master': '', '_save_checkpoints_steps': 2000, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': 2020, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 500, '_model_dir': './model/iris_DNN', '_global_id_in_cluster': 0, '_save_summary_steps': 100}


In [8]:
# Train the Model.
#classifier.train(
#  input_fn=make_train_input_fn(train_x, train_y, 100),
#  steps=20000)

train_spec = tf.estimator.TrainSpec(
  input_fn=make_train_input_fn(train_x, train_y, 100),
  max_steps=20000)

eval_spec = tf.estimator.EvalSpec(
  input_fn=make_eval_input_fn(test_x, test_y),
  steps=None,          # use complete test set
  start_delay_secs=0,  # start immediately
  throttle_secs=10)    # minimum delay between evaluations

tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 10 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./model/iris_DNN/model.ckpt.
INFO:tensorflow:loss = 125.53413, step = 1
INFO:tensorflow:global_step/sec: 479.369
INFO:tensorflow:loss = 7.2349, step = 501 (1.046 sec)
INFO:tensorflow:global_step/sec: 492.324
INFO:tensorflow:loss = 6.1838913, step = 1001 (1.015 sec)
INFO:tensorflow:global_step/sec: 488.15
INFO:tensorflow:loss = 5.1144276, step = 1501 (1.025 sec)
INFO:tensorflow:Saving checkpoints for 2001 into ./model/iris_DNN/model.ckpt.
INFO:tensorflow:global_step/sec: 559.895
INFO:tensorflow:loss =

INFO:tensorflow:Saving checkpoints for 20000 into ./model/iris_DNN/model.ckpt.
INFO:tensorflow:Loss for final step: 1.2145597.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-12-19:45:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/iris_DNN/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-12-19:45:44
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.96666664, average_loss = 0.11396465, global_step = 20000, loss = 3.4189394


In [9]:
# Evaluate the model.
eval_result = classifier.evaluate(
  input_fn=make_eval_input_fn(test_x, test_y))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-12-19:45:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/iris_DNN/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-12-19:45:44
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.96666664, average_loss = 0.11396465, global_step = 20000, loss = 3.4189394

Test set accuracy: 0.967



In [10]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
  'SepalLength': [5.1, 5.9, 6.9],
  'SepalWidth': [3.3, 3.0, 3.1],
  'PetalLength': [1.7, 4.2, 5.4],
  'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(input_fn=make_eval_input_fn(predict_x))

template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(predictions, expected):
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print(template.format(SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/iris_DNN/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (100.0%), expected "Setosa"

Prediction is "Versicolor" (100.0%), expected "Versicolor"

Prediction is "Virginica" (100.0%), expected "Virginica"
